In [14]:
# !pip install googletrans==4.0.0-rc1
# !pip install pyspark py4j
# !pip install findspark

In [7]:
from googletrans import Translator
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import pandas as pd
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession

In [17]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("PySpark - ContandoPalavras").getOrCreate()
sc = spark.sparkContext

spark

In [161]:
df = spark.read.csv("C:/Users/cedua/CDIA - PUCSP/PROJETO PySpark (SAVINO)/Contador-de-palavras---PySpark/data/fitnessandpower_data.csv", header=True, inferSchema=True)
df.show()

+--------------------+
|       articles_text|
+--------------------+
|When you want to ...|
|These powders com...|
|ARE THERE ANY DIF...|
|Yes! Whey protein...|
|This means that t...|
|DO THEY HAVE THE ...|
|No! Whey protein ...|
|DO THEY TASTE THE...|
|Nope! Whey protei...|
|DO THEY CONTAIN T...|
|No! 100 grams of ...|
|ARE THEY ABSORBED...|
|No. Whey protein ...|
|          All in all|
|If you want to ha...|
|Not as well as th...|
|For the latest ne...|
|            Facebook|
|             Twitter|
|           Pinterest|
+--------------------+
only showing top 20 rows



pyspark.sql.dataframe.DataFrame

In [26]:
########################################################################################################################

In [20]:
#### EXEMPLO 1 - MAP REDUCE
dados=["B","B","B","C","C","C","D","D","D D"]

input_file=sc.parallelize(dados,3)
counts = input_file.map(lambda word:(word, 1)) \
                     .reduceByKey(lambda a, b: a + b)

result=counts.collect()
print(result) 
df=pd.DataFrame(result)
df

[('B', 3), ('C', 3), ('D', 2), ('D D', 1)]


,0,1
0,B,3
1,C,3
2,D,2
3,D D,1


In [22]:
#### EXEMPLO 2 - MAP REDUCE
dados=["B","B","B","C","C","C","D"]
item=["B","D"]
input_file=sc.parallelize(dados,3)
counts = (input_file.map(lambda word:(word, 1) if (word in item) else (word,0)) 
                     .reduceByKey(lambda a, b: a + b))

result=counts.collect()
print(result)
df=pd.DataFrame(result)
# df.query("qtde>0")
df

[('B', 3), ('C', 0), ('D', 1)]


,0,1
0,B,3
1,C,0
2,D,1


In [27]:
#########################################################################################################################

In [153]:
def translate_text(text):
    translator = Translator()
    translation = translator.translate(text, dest="pt")
    return translation.text

In [193]:
# Cria a função UDF (User Defined Function) para aplicar a tradução
translate_udf = udf(translate_text, StringType())

# Aplica a tradução no DataFrame usando a função UDF
translated_df = df.withColumn("translated_text", translate_udf(df["articles_text"]))
translated_df = translated_df.drop("articles_text")

# Mostra o DataFrame com os textos traduzidos
translated_df.show()


+--------------------+
|     translated_text|
+--------------------+
|Quando você desej...|
|Esses pós vêm em ...|
|Existem diferença...|
|Sim!O isolado de ...|
|Isso significa qu...|
|Eles têm o mesmo ...|
|Não!O concentrado...|
|Eles têm o gosto ...|
|Não!O isolado de ...|
|Eles contêm a mes...|
|Não!100 gramas de...|
|Eles são absorvid...|
|Não. O isolado de...|
|             Contudo|
|Se você quiser te...|
|Não tão bem quant...|
|Para as últimas n...|
|            Facebook|
|             Twitter|
|           Pinterest|
+--------------------+
only showing top 20 rows



In [195]:
from pyspark.sql.functions import split
from pyspark.sql.functions import explode, col
from pyspark.sql.functions import regexp_extract
from pyspark.sql.functions import lower

# def split_by_space(df):
#     #spliting the df by space " " 
#     df1 = df.select(
#         split(df['translated_text'], ' ').alias('words')
#     ).show()

#     return df1.show()

# def drop_blank_spaces(df1):
#     #droping blank spaces
#     df2 = df1.select(
#     explode(col("words")).alias("words2")
#     )

#     df3 = df2.where(
#     col('words2') != ''
#     )
#     return df3.show()

# def remove_punc(df3):
#     #removing punctuation
#     df4 = df3.select(
#     regexp_extract(
#         col('words2'), #cleaning words2 column
#         '[A-z]+', #selecting all the words
#         0).alias('words3')
#     )
#     return df4

# def lower_case_words(df4):
#     #every word to lower case
#     df5 = df4.select(
#     lower(
#         col('words3')
#     ).alias('words4')
#     )
#     return df5

In [194]:
#spliting the df by space " " 
translated_df = translated_df.select(
split(translated_df['translated_text'], ' ').alias('words')
)

translated_df.show()
type(translated_df)

+--------------------+
|               words|
+--------------------+
|[Quando, você, de...|
|[Esses, pós, vêm,...|
|[Existem, diferen...|
|[Sim!O, isolado, ...|
|[Isso, significa,...|
|[Eles, têm, o, me...|
|[Não!O, concentra...|
|[Eles, têm, o, go...|
|[Não!O, isolado, ...|
|[Eles, contêm, a,...|
|[Não!100, gramas,...|
|[Eles, são, absor...|
|[Não., O, isolado...|
|           [Contudo]|
|[Se, você, quiser...|
|[Não, tão, bem, q...|
|[Para, as, última...|
|          [Facebook]|
|           [Twitter]|
|         [Pinterest]|
+--------------------+
only showing top 20 rows



pyspark.sql.dataframe.DataFrame

In [196]:
#droping blank spaces
translated_df = translated_df.select(
explode(col("words")).alias("words2")
)

translated_df.show()
type(translated_df)

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "c:\Users\cedua\Anaconda3\lib\socket.py", line 707, in readinto
    raise
socket.timeout: timed out
